[Reference](https://python.plainenglish.io/multiprocessing-itself-isnt-enough-9a62ce8f5a67)

In [2]:
from datetime import datetime
import multiprocessing as mp
import psutil
from collections import defaultdict

start_time = datetime.now()

def loop(core, start, stop):
    """
    Function that sums each n in a range to its previous n.
    Remember the for loop isn't necessary and it is here just to make the computation time longer.
    """
    proc = psutil.Process()
    proc.cpu_affinity([core])
    for n in range(start, stop, 1):
        x = (n*(n+1))/2
    return x
    
def cores():
    """
    create a list with every virtual core number in the system
    """
    cores = []
    for c in range(psutil.cpu_count()):
        cores.append(c)
    return cores
    
def block_size(r, cores):
    """
    Define each block size (b) to be processed.
    This function returns a dictionary containing the core as key
        and a list as value, which contains the start and stop limits for the "for loop".
    PS: the range for the last core can be wider sometimes. It happens because the floor division of a range by the number of cores, sometimes evaluates to an integer that multiplied to the number of cores, would result in a lower range (remainder) - this function takes care of it too.
    """
    blocks = defaultdict(list)
    b = r // len(cores)
    for c in cores:
        if b == 0:
            blocks[c] = [0, r]
            return blocks
        else:
            if b*(c+1) > r:
                stop = r
            else:
                stop = b*(c+1)
            if c == cores[-1]:
                stop = b*(c+1) + (r - b*(c+1))
            blocks[c] = [b*(c), stop]
    return blocks
    
ranges = [100000000, 200000000, 300000000]
cores = cores()

if __name__ == "__main__":    
    
    results = defaultdict(dict)
    with mp.Pool() as pool:
        for r in ranges:
            # split the full range into smaller ranges per virtual core
            blocks = block_size(r, cores)
            for core in blocks:
                range_index = f'{ranges.index(r)}'
                start = blocks[core][0]
                stop = blocks[core][1]
                # call the function loop for each smaller range of each range
                p = pool.apply_async(func=loop, args=(core, start, stop,))
                # append the returned objects and organize them by range index and core
                results[range_index][core] = p
        pool.close()
        pool.join()    
        
    result = 0
    for range_index in results:
        for core in results[range_index]:
            x = results[range_index][core].get()
        # remember we only care about the last result
        # so we sum the last result obtained for each range in ranges
        result = result+x    
        
    print(f"Result: {result}")
    print('')
    print(f"Time spent: {datetime.now() - start_time}")

Result: 6.99999997e+16

Time spent: 0:01:51.059016
